<a href="https://colab.research.google.com/github/ajayvallabh/PytorchTutorial/blob/main/CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.utils import make_grid

import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
%matplotlib inline

In [5]:
# Convert MNIST Image Files into a Tensor of 4-Dimensions. (# of images, Height, Width, Color Channels)
transform = transforms.ToTensor()

In [6]:
# Train Data
train_data = datasets.MNIST(root='/cnn_data', train = True, download=True, transform=transform)

100%|██████████| 9.91M/9.91M [00:00<00:00, 55.3MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 1.81MB/s]
100%|██████████| 1.65M/1.65M [00:00<00:00, 14.5MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 5.01MB/s]


In [7]:
# Test Data
test_data = datasets.MNIST(root='/cnn_data', train = False, download=True, transform=transform)

In [8]:
train_data

Dataset MNIST
    Number of datapoints: 60000
    Root location: /cnn_data
    Split: Train
    StandardTransform
Transform: ToTensor()

In [9]:
test_data

Dataset MNIST
    Number of datapoints: 10000
    Root location: /cnn_data
    Split: Test
    StandardTransform
Transform: ToTensor()

In [10]:
  # Create a small batch size for images...
  train_loader = DataLoader(train_data, batch_size=10, shuffle= True)
  test_loader = DataLoader(test_data, batch_size=10, shuffle= False)


In [11]:
# Define CNN Model
# Describe convolutional layer and what it's doing (2 convolutional layers)
# This is just an example
conv1 = nn.Conv2d(1, 6, 3, 1) # input size of one, first convolutional layer we want six filters or output feature maps, kernel size 3 by 3, step it 1 at a time
conv2 = nn.Conv2d(6, 16, 3, 1) # number of input is 6


In [12]:
# Grab 1 MNIST record/image
for i, (X_train, y_train) in enumerate(train_data):
  break

In [13]:
X_train.shape

torch.Size([1, 28, 28])

In [14]:
x = X_train.view(1, 1, 28, 28)

In [15]:
# Perform our first convolutional
x = F.relu(conv1(x)) # Rectified linear Unit for our activation function

In [16]:
# 1 single image, 6 is the filters we asked for, 26x26
x.shape

torch.Size([1, 6, 26, 26])

In [17]:
# pass thru pooling layer
x = F.max_pool2d(x,2,2) # kernal of 2 and stride of 2

In [18]:
x.shape  # 26/2 =13

torch.Size([1, 6, 13, 13])

In [19]:
# Do our second convolutional layer
x = F.relu(conv2(x))

In [20]:
x.shape # Again, we did not set padding so we lose 2 pixels around the outside of the image

torch.Size([1, 16, 11, 11])

In [21]:
# Pooling layer
x = F.max_pool2d(x,2,2)

In [22]:
x.shape # 11/2 =5.5 but we have to round down, because you can't invent data to round up

torch.Size([1, 16, 5, 5])

# CNN model

In [23]:
# Model class
class ConvolutionalNetwork(nn.Module):
  def __init__(self):
    super().__init__()
    self.conv1 = nn.Conv2d(1,6,3,1) # input, output, kernel matrix size, stride=1
    self.conv2 = nn.Conv2d(6,16,3,1)
    # Fully connected  neural layer
    self.fc1 = nn.Linear(5*5*16, 120)
    self.fc2 = nn.Linear(120, 84)
    self.fc3 = nn.Linear(84, 10)

  def forward(self, X):
    X = F.relu(self.conv1(X))
    X = F.max_pool2d(X,2,2) # 2*2 kernal and stride 2
    # Second Pass
    X = F.relu(self.conv2(X))
    X = F.max_pool2d(X,2,2) # 2*2 kernal and stride

    # Re-View to flatten it out
    X = X.view(-1, 16*5*5) # negative one so that we can vary the batch size

    # Fully Connected Layers
    X = F.relu(self.fc1(X))
    X = F.relu(self.fc2(X))
    X = self.fc3(X)
    return F.log_softmax(X, dim=1)


In [24]:
# Create an Instance of our model
torch.manual_seed(41)
model = ConvolutionalNetwork()
model

ConvolutionalNetwork(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)

In [25]:
# Loss Function Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001) # Smaller the learning rate, longer it will take to train

In [ ]:
import time
start_time = time.time()

# Create Variables to tracks things
epochs = 5
train_losses = []
test_losses = []
train_correct = []
test_correct = []

# For Loop of Epochs
for i in range(epochs):
  trn_corr = 0
  tst_corr = 0

  # Train
  for b,(X_train, y_train) in enumerate(train_loader):
    b+=1 # start our batches at 1
    y_pred = model(X_train) # get predicted the values from the training set. Not flattened 2d
    loss = criterion(y_pred, y_train) # how off are we? Compare the predictions to correct answers in y_train

    predicted = torch.max(y_pred.data,1)[1] # add up the number of correct predictions.
    batch_corr = (predicted == y_train).sum() # how many we got correct from this batch. True = 1, False = 0, sum those up
    trn_corr += batch_corr # keep track as we go along in training

    # Update our parameters
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # Print out some results
    if b%600 == 0:
      print(f'Epoch: {i} Batch: {b} Loss: {loss.item()}')

    train_losses.append(loss)
    train_correct.append(trn_corr)

    # Test
    with torch.no_grad():  # No gradient so we don't update our weight and biases with test data
      for b,(X_test, y_test) in enumerate(test_loader):
        y_val = model(X_test)
        predicted = torch.max(y_val.data, 1)[1] # adding up correct predictions
        tst_corr += (predicted == y_test).sum() # T=1 F=0 and sum away


current_time = time.time()
total = current_time - start_time
print(f'Training Took: {total/60} minutes!')

In [ ]:
# Graph the loss at epoch
train_losses = [tl.item() for tl in train_losses]
plt.plot(train_losses, label="Training Loss")
plt.plot(test_losses, label="Validataion Loss")
plt.title("Loss at Epoch")
plt.legend()

In [ ]:
# Graph the accuracy at the end of epoch
plt.plot([t/600 for t in train_correct], label="Training Accuracy")
plt.plot([t/100 for t in test_correct], label="Validation Accuracy")
plt.title("Accuracy at the end of each Epoch")
plt.legend()

In [ ]:
test_load_everything = DataLoader(test_data, batch_size=10000, shuffle=False)

In [ ]:
with torch.no_grad():
  correct = 0
  for X_test, y_test in test_load_everything:
    y_val = model(X_test)
    predicted = torch.max(y_val, 1)[1]
    correct += (predicted == y_test).sum()


In [ ]:
# Did for correct
correct.item()/len(test_data)*100